In [ ]:
import math
import pandas as pd
import numpy as np
import scipy.optimize as optimize
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from datetime import datetime

def disFactInpolDate(discFacs, maturityYears, interpolatedPeriods ):
    #print(" Enter: disFactInpolDate")
    discFactors =  list(discFacs)   
    periods = [x * 2 for x in maturityYears] 
    intPolStrips = interp1d(periods, discFactors, kind='cubic')
    discFact = intPolStrips(interpolatedPeriods)
    #print(" Exit: disFactInpolDate")
    return discFact


def swapInterPolate(valueData, maturityYears, interpolatedPeriods , tipsCouponRate):
    #print(" Enter: swapInterPolate")
    swapPeriods = [x * 2 for x in maturityYears]
    inflationSwap = []
    swapRates =  list(valueData)
    for i in range(len(swapPeriods)):
        inflationSwap.append(tipsCouponRate/2 * math.pow((1+swapRates[i]/200), swapPeriods[i]))
    intPolSwap = interp1d(swapPeriods, inflationSwap, kind='cubic')
    intPolSwapPrice = intPolSwap(interpolatedPeriods)
    #print(" Exit: swapInterPolate")
    return intPolSwapPrice

def funcReadSwapData():
    #print(" Enter: funcReadSwapData")
    df = pd.read_excel('swap.xlsx', sheet_name='swaps')
    matYrs = df.columns.tolist()
    listDates =list(df.index)  
    n =  len(listDates)
    drDict = {}
    for i in range(n):
        t = listDates[i]
        if(type(t)== str):
            drDict[t] = df.iloc[i,0:16]
        else:
            string = '{:%m/%d/%Y}'.format(t)
            drDict[string] = df.iloc[i,0:16]
    #print(" Exit: funcReadSwapData")
    return drDict, matYrs
    
def funcReadStripsData():
    #print(" Enter: funcReadStripsData")
    df = pd.read_excel('disFactors.xlsx', sheet_name='factors')
    maturityYears = df.columns.tolist()
    listDates =list(df.index)  
    n =  len(listDates)
    drDict = {}
    for i in range(n):
        t = listDates[i]
        if(type(t)== str):
            st = datetime.strptime(t, "%m/%d/%Y")
            string = '{:%m/%d/%Y}'.format(st)
            drDict[string] = df.iloc[i,0:15]
        else:
            string = '{:%m/%d/%Y}'.format(t)
            drDict[string] = df.iloc[i,0:15]
    #print(" Exit: funcReadStripsData")
    return drDict, maturityYears

def matchTreasuryTipsDate():
    #print(" Enter: matchTreasuryTipsDate")
    df = pd.read_excel('pair2.xlsx', sheet_name='Sheet1')
    df.head()

    treasuryCouponRate = df.columns.tolist()[1]
    tipsCouponRate = df.columns.tolist()[3]

    trsMatur = df.iloc[0,1]
    tpsMatur = df.iloc[0,3]
    pairDictionTemp = {}
    pairDictionConcise = {}

    for i in range(2, len(df.iloc[:,1])):
        if(math.isnan(df.iloc[i,1])):
            print("error in parsing treasury data")
        else:
            t = df.iloc[i,0]
            if(type(t)== str):
                st = datetime.strptime(t, "%m/%d/%Y")
                string = '{:%m/%d/%Y}'.format(st)
            else:
                string = '{:%m/%d/%Y}'.format(df.iloc[i,0])
            pairDictionTemp[string] = df.iloc[i,1]
        
    for i in range(2, len(df.iloc[:,3])):
   
        if(math.isnan(df.iloc[i,3]) ):
            print("error in parsing tips data ")
        else:
            t = df.iloc[i,0]
            
            if(type(t)== str):
                st = datetime.strptime(t, "%m/%d/%Y")
                string = '{:%m/%d/%Y}'.format(st)
            else:
                string = '{:%m/%d/%Y}'.format(df.iloc[i,0])
                
            if string in pairDictionTemp:
                pairDictionConcise[string] = [pairDictionTemp[string], df.iloc[i,3]]
            else:
                print("tips treasury date mismatch")
    #print(" Exit: matchTreasuryTipsDate")
    return treasuryCouponRate, tipsCouponRate, trsMatur, tpsMatur, pairDictionConcise

def getIntPolPeriod(MaturityDate, StartDate):
    #print(" Enter: getIntPolPeriod")
    if(type(MaturityDate)== str):
        print("came")
        mat = datetime.strptime(MaturityDate, "%m/%d/%Y").timestamp()   
    else:
        mat = MaturityDate.timestamp()
    if(type(StartDate)== str):
        print("came")
        start = datetime.strptime(StartDate, "%m/%d/%Y").timestamp()   
    else:
        start = StartDate.timestamp()    
    counter = 0
    while(mat > start):
        counter = counter + 1
        mat =  mat - 15552000
    days =  int((start - mat)/86400)
    interpolatedPeriods = []
    for i in range(counter):
        interpolatedPeriods.append(days/180 + i)
    #print(" Exit: getIntPolPeriod")
    return interpolatedPeriods

def getSynBondPrice(newSwapRates, newDiscFactors,trsCouponRate, tipsCouponRate):
    #print(" Enter: getSynBondPrice")
    synBondPrice = 0
    for i in range(len(newSwapRates)):
        if i == (len(newSwapRates) -1 ):
            temp = (100+tipsCouponRate/2)* (newSwapRates[i]/(tipsCouponRate/2))
            val = (100 + trsCouponRate/2) -  temp
            synBondPrice = synBondPrice + val*newDiscFactors[i]
        else:
            val = trsCouponRate/2 -  newSwapRates[i]
            synBondPrice = synBondPrice + val*newDiscFactors[i]
    #print(" Exit: getSynBondPrice")
    return synBondPrice


def funMatMismatchYTM(price, matArr, coup, freq, guess):
    ytm_func = lambda y: sum([coup/((1+y/freq)**t) for t in matArr]) + 100 / ((1+y/freq)** matArr[-1]) - price    
    return optimize.newton(ytm_func, guess)

def calBondPriceYTM(coup, freq, matArr, ytm ):
    price = sum([coup/((1+ytm/freq)**t) for t in matArr]) + 100 / ((1+ytm/freq)** matArr[-1])
    return price

def main():
    stripsData, matStrips = funcReadStripsData()
    swapData, matSwaps = funcReadSwapData()

    trsCouponRate, tipsCouponRate, trsMatur, tpsMatur, matchDateTpsTrs = matchTreasuryTipsDate()
        
    printString = "Treasury "+ str(trsMatur)+": "+ str(trsCouponRate)+"-- Tips "+ str(tpsMatur)+": "+ str(tipsCouponRate)
    arbitrageArray = []
    timeArr = []
    i = 1
    for key in matchDateTpsTrs:
        intPolTrs = getIntPolPeriod(trsMatur, key)
        intPolTps = getIntPolPeriod(tpsMatur, key)
               
        if key in stripsData:
            newDiscFactors = disFactInpolDate(stripsData[key], matStrips, intPolTps)
            newSwapRates = swapInterPolate(swapData[key], matSwaps, intPolTps, tipsCouponRate)

            synBondPrice = getSynBondPrice(newSwapRates, newDiscFactors,trsCouponRate, tipsCouponRate) + matchDateTpsTrs[key][1]
            ytm = funMatMismatchYTM(synBondPrice, intPolTps, trsCouponRate/2, 2, 0)
            if(ytm < 0.10):
                synBondPriceYtm = calBondPriceYTM(trsCouponRate/2, 2, intPolTrs, ytm )
                tinyArbitrage = matchDateTpsTrs[key][0] -synBondPriceYtm
                #print(matchDateTpsTrs[key][0])
                #print(key+" " + str(synBondPrice) + " "+ str(ytm) + " "+ str(synBondPriceYtm)+"arbit:" +str(tinyArbitrage ))
                arbitrageArray.append(tinyArbitrage)
                timeArr.append(i)
                i = i+1
    print(printString)       
    print("Max:" + str(max(arbitrageArray)))  
    print("Min:" + str(min(arbitrageArray)))
    print("Mean:" + str(np.mean(arbitrageArray)))  
    print("Standard Deviation:" + str(np.std(arbitrageArray)))
    plt.plot(timeArr, arbitrageArray, 'b--', label = printString)
    plt.ylabel('arbitrage')
    plt.xlabel('time')
    plt.legend()
    plt.show()
        


   
        
        
        
if __name__=='__main__':
    main()